In [1]:
import numpy as np
import QuantLib as ql
from src.QlMarket import QlMarket
from src.QlStockPricer import QlStockPricer
from src.QlVanillaOption import QlVanillaOption


In [2]:
S0 = 100
strike = 100
r = 0.05
sigma = 0.3
T=1

In [3]:
start_date = ql.Date(1, 1, 2023)
qlMarket = QlMarket(init_date=start_date, init_risk_free_rate=r)
start_date = qlMarket.init_date
end_date = qlMarket.yearFractionToDate(start_date, 1)
start_date, end_date

(Date(3,1,2023), Date(10,1,2024))

In [4]:
qlMarket.calendar

<QuantLib.QuantLib.HongKong; proxy of <Swig Object of type 'QuantLib::HongKong *' at 0x10a9d4de0> >

In [5]:
stock_1 = QlStockPricer(QlMarket=qlMarket, init_stock_price=S0)
stock_1.using_Black_Scholes_model(sigma=sigma)


Next step, you need set model type
set new model: Black Scholes model with sigma: 0.3


In [6]:
stock_1.day_counter.yearFraction(start_date, end_date)

1.0

In [7]:
option_call = QlVanillaOption.init_from_price_and_date(
    strike_price=strike,
    end_date=end_date,
    QlStockPricer = stock_1,
    option_type=ql.Option.Call,
    engine = ql.AnalyticEuropeanEngine
)
option_put = QlVanillaOption.init_from_price_and_date(
    strike_price=strike,
    end_date=end_date,
    QlStockPricer = stock_1,
    option_type=ql.Option.Put,
    engine = ql.AnalyticEuropeanEngine
)
print(f'call: {option_call.NPV()}')
print(f'put: {option_put.NPV()}')
print(f'call - put : {option_call.NPV() - option_put.NPV()}')

call: 14.231254785985845
put: 9.354197236057235
call - put : 4.87705754992861


## test if fulfills put-call parity


In [8]:
print(option_call.NPV() - option_put.NPV() - (S0 - strike * np.exp(-r * T)))


1.5987211554602254e-14


In [9]:
from scipy.stats import norm

def bs_option(S0, strike, T, r, sigma, type):
    '''
    Black-Scholes option pricing formula

    Parameters
    ----------
    S0 : float
        Initial value of the underlying
    strike : float
        Strike price of the option
    T : float
        Time to maturity of the option
    r : float
        Risk-free interest rate
    sigma : float
        Volatility of the underlying
    type : str
        Type of option, either 'call' or 'put'
    '''
    d1 = (np.log(S0/strike) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if type == 'call':
        return S0 * norm.cdf(d1) - strike * np.exp(-r * T) * norm.cdf(d2)
    elif type == 'put':
        return strike * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    else:
        raise ValueError('Option type must be either "call" or "put"')


In [10]:
# test if fulfills put-call parity
call = bs_option(S0, strike, T, r, sigma, 'call')
put = bs_option(S0, strike, T, r, sigma, 'put')

print(f'call: {call}')
print(f'put: {put}')
print(f'call - put : {call - put}')
print(call - put - (S0 - strike * np.exp(-r * T)))

call: 14.231254785985819
put: 9.354197236057232
call - put : 4.877057549928587
-7.105427357601002e-15


In [11]:
call

14.231254785985819

In [12]:
put

9.354197236057232